# Settings

In [51]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
import holoviews as hv
import base as b
import talib as ta
import hvplot.pandas  # noqa
from functools import cache
from logger_settings import logger
from constants import *
from bokeh.plotting import figure, show, output_notebook
import json
output_notebook()

files = ["FINNIFTY-27AUG-23350-CE.json", "FINNIFTY-27AUG-23750-PE.json", "FINNIFTY-27AUG.json"]

with open(files[0]) as f2:
    fincall = pd.DataFrame(json.loads(f2.read())["data"])
with open(files[2]) as f1:
    findf = pd.DataFrame(json.loads(f1.read())["data"])
with open(files[1]) as f3:
    finput = pd.DataFrame(json.loads(f3.read())["data"])

for idx, row in findf.iterrows():
    findf.at[idx, "call_close"] = fincall.iloc[idx].c
    findf.at[idx, "put_close"] = finput.iloc[idx].c
    findf.at[idx, "call_volume"] = fincall.iloc[idx].v / fincall.iloc[idx-1].v
    findf.at[idx, "put_volume"] = finput.iloc[idx].v / finput.iloc[idx-1].v

findf["call_close_diff"] = findf.call_close.diff()
findf["fin_close_diff"] = findf.c.diff()
findf["put_close_diff"] = findf.put_close.diff()
findf["call_expected_ch"] = findf.fin_close_diff / 2
findf["put_expected_ch"] = - findf.fin_close_diff / 2
findf["call_ex_diff"] = findf.call_close_diff / findf.call_expected_ch
findf["put_ex_diff"] = findf.put_close_diff / findf.put_expected_ch
findf["total_ex_diff"] = (findf.call_ex_diff - findf.put_ex_diff) * (findf.call_volume.quantile() + findf.put_volume)
# findf["total_ex_diff"] = findf.total_ex_diff.rank(pct=True)

Loading BokehJS ...

/tmp/ipykernel_88954/1934478664.py:29: RuntimeWarning: divide by zero encountered in scalar divide
  findf.at[idx, "put_volume"] = finput.iloc[idx].v / finput.iloc[idx-1].v
/tmp/ipykernel_88954/1934478664.py:29: RuntimeWarning: invalid value encountered in scalar divide
  findf.at[idx, "put_volume"] = finput.iloc[idx].v / finput.iloc[idx-1].v
/tmp/ipykernel_88954/1934478664.py:28: RuntimeWarning: divide by zero encountered in scalar divide
  findf.at[idx, "call_volume"] = fincall.iloc[idx].v / fincall.iloc[idx-1].v


In [55]:
# findf.hvplot.line(x='Time', y=['call_ex_diff', "put_ex_diff", "total_ex_diff"], width=2000, height=1000, tools=['vline', 'crosshair'], 
#                   # hover_cols=['last_price', 'volume', 'index']
#                  )

findf.hvplot.line(x='Time', y=["call_close", "put_close", "c"], width=2000, height=1000, tools=['vline', 'crosshair'], 
                  # hover_cols=['last_price', 'volume', 'index']
                 )
findf.hvplot.line(x='Time', y=["o", "h", "l", "c"], width=2000, height=1000, tools=['vline', 'crosshair'], 
                  # hover_cols=['last_price', 'volume', 'index']
                 )

# findf.iloc[1900:2000].hvplot.line(x='Time', y=['call_ex_diff', "put_ex_diff", "total_ex_diff"], width=2000, height=1000, tools=['vline', 'crosshair'], 
#                   # hover_cols=['last_price', 'volume', 'index']
#                  )

# findf.iloc[1900:2000].hvplot.line(x='Time', y=['call_ex_diff', "put_ex_diff", "total_ex_diff"], width=2000, height=1000, tools=['vline', 'crosshair'], 
#                   # hover_cols=['last_price', 'volume', 'index']
#                  )

:NdOverlay   [Variable]
   :Curve   [Time]   (value)

In [ ]:
test_date = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
TEST_START = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2024-01-10", "%Y-%m-%d")

train_dates, test_dates = ut.get_date_range(start_date=TEST_START, end_date=TEST_END, symbol=SYMBOL, interval=INTERVAL, exchange=EXCHANGE)
train_dates["expiry"] = pd.NA
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)


from typing import Dict

settings = {
    # Parameters
    "VOLUME_CUT_OFF": 0.64,
    "COOLDOWN": 2, # 2 candles
    "TP": 0.05, # 5%
    # Order Settings
    "quantity": 25,
}

Loading BokehJS ...

# Strategy

In [64]:
class PeakStrategy(b.Strategy):
    def __init__(self, instrument: "Instrument", settings: Dict, meta: Dict ={}):
        super().__init__(instrument, settings)
        self.t1_ticks: pd.DataFrame | None = None
        self.t2_ticks: pd.DataFrame | None = None
        self.meta = meta
        self.om = b.OrderManager()

    def calculate_data(self):
        self.t1_ticks['is_green'] = self.t1_ticks.close >= self.t1_ticks.open
        self.t1_ticks['is_red'] = ~self.t1_ticks.is_green
        self.t1_ticks['vratio'] = self.t1_ticks.volume /  self.t1_ticks.volume.shift(1)
        self.t2_ticks['is_green'] = self.t2_ticks.close >= self.t2_ticks.open
        self.t2_ticks['is_red'] = ~self.t2_ticks.is_green
        self.t2_ticks['vratio'] = self.t2_ticks.volume /  self.t2_ticks.volume.shift(1)
        self.t1_ticks['rsi'] = ta.RSI(self.t1_ticks.close, timeperiod=28)
        self.t2_ticks['rsi'] = ta.RSI(self.t2_ticks.close, timeperiod=28)

    def entry_conditions(self):
        def match_conditions(df):
            last, second_last = df.iloc[-1], df.iloc[-2]
            if second_last.is_red and last.is_green and last.vratio <= self.VOLUME_CUT_OFF and second_last.rsi < 35 and last.rsi < 45:
                return True
            return False

        if self.t1_ticks.shape[0] < 2:
            return 0

        matches = match_conditions(self.t1_ticks)
        if matches:
            return 1
        matches = match_conditions(self.t2_ticks)
        if matches:
            return 2
        return 0

    def exit_conditions(self, order) -> bool:
        if order.meta['type'] == 1:
            tdf = self.t1_ticks
        else:
            tdf = self.t2_ticks
        last_tick = tdf.iloc[-1]
        if last_tick.name.hour == 15 and last_tick.name.minute == 28:
            return True, last_tick.close
        openp = last_tick.low
        closep = last_tick.close

        for order in self.om.orders:
            if last_tick.name <= (order.created_at + dt.timedelta(minutes=self.COOLDOWN)):
                # 1 minute cooldown time
                continue
            # if tdf.iloc[-2:].loc[~tdf.is_green].shape[0] == 2:
            #     # Two consecutive red candles
            #     return True, last_tick.close
            # if (not last_tick.is_green) and last_tick.vratio <= self.VOLUME_CUT_OFF:
            #     # Red candle formed with half volume
            #     return True, last_tick.close
            if (order.limit_price - openp) >= order.meta['sl']:
                return True, order.limit_price - order.meta['sl']
            if (openp - order.limit_price) >= order.meta['tp']:
                return True, order.limit_price + order.meta['tp']
            if (closep - order.limit_price) >= order.meta['tp']:
                return True, order.limit_price + order.meta['tp']
            if (order.limit_price - closep) >= order.meta['sl']:
                return True, order.limit_price - order.meta['sl']
        return False, None

    def _process_tick(self, t1_ticks, t2_ticks):
        self.t1_ticks = t1_ticks
        self.t2_ticks = t2_ticks
        # self.t1_ticks["timestamp"] = self.t1_ticks.index
        # self.t2_ticks["timestamp"] = self.t2_ticks.index
        self.calculate_data()

    def next(self, t1_ticks, t2_ticks):
        self._process_tick(t1_ticks=t1_ticks, t2_ticks=t2_ticks)
        if not self.om.has_intrade_orders():
            entry = self.entry_conditions()
            if entry == 1:
                target = self.t1_ticks.iloc[-1].close * self.TP
                stoploss = target
                order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.t1_ticks.iloc[-1].close, created_at=self.t1_ticks.iloc[-1].name,
                                quantity=self.quantity, exchange_order_id=None, 
                                meta={'atm_strike': self.meta['atm_strike'], 'tp': target, 'sl': stoploss, "type": entry})
                self.om.place_order(order)
            elif entry == 2:
                target = self.t2_ticks.iloc[-1].close * self.TP
                stoploss = target
                order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.t2_ticks.iloc[-1].close, created_at=self.t2_ticks.iloc[-1].name,
                                quantity=self.quantity, exchange_order_id=None, 
                                meta={'atm_strike': self.meta['atm_strike'], 'tp': target, 'sl': stoploss, "type": entry})
                self.om.place_order(order)
        if self.om.has_intrade_orders():
            for order in self.om.orders:
                is_exit, eprice = self.exit_conditions(order)
                if is_exit:
                    self.om.square_off_order(order=order, index=self.t1_ticks.iloc[-1].name, last_price=eprice)

# Execution

In [65]:
instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")
ps = PeakStrategy(instrument=instrument, settings=settings)

for index, row in train_dates.iterrows():
    nifty_candles = ut.get_data(symbol=SYMBOL, date=index.date(), interval=INTERVAL, exchange=EXCHANGE)
    for i in range(nifty_candles.shape[0]):
        # atm_strike = ut.get_atm_strike(nifty_candles.iloc[i].close)
        ps.meta['atm_strike'] = ut.get_atm_strike(nifty_candles.iloc[i].close)
        t1_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=ps.meta['atm_strike'], option_type=OPTION_TYPE_CALL)
        t2_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=ps.meta['atm_strike'], option_type=OPTION_TYPE_PUT)
        ps.next(t1_ticks=t1_df.iloc[:i+1].copy(), t2_ticks=t2_df.iloc[:i+1].copy())
    break

# Results

In [37]:
pd.set_option("display.max_rows", None)

ps.t1_ticks.head(200)

,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4,is_green,is_red,vratio,rsi
date_time,,,,,,,,,,,,,
2024-01-01 09:15:00,150.00,164.20,136.00,137.45,724000,3413000,140.85,137.15,137.50,False,True,NaN,NaN
2024-01-01 09:16:00,138.35,143.00,135.75,137.65,668950,3413000,139.99,137.30,137.65,False,True,0.923964,NaN
2024-01-01 09:17:00,137.60,141.35,136.55,136.80,442800,3757500,139.66,136.45,136.80,False,True,0.661933,NaN
2024-01-01 09:18:00,137.80,139.05,131.75,133.50,551450,3757500,138.86,133.20,133.50,False,True,1.245370,NaN
2024-01-01 09:19:00,134.60,136.05,130.00,130.35,458650,3757500,137.98,130.10,130.35,False,True,0.831716,NaN
2024-01-01 09:20:00,130.30,133.20,127.60,130.65,691900,4235100,136.57,130.50,130.70,True,False,1.508558,NaN
2024-01-01 09:21:00,131.45,134.30,129.70,133.00,554200,4235100,135.94,133.05,133.25,True,False,0.800983,NaN
2024-01-01 09:22:00,133.75,137.20,133.75,136.35,563350,4235100,135.95,136.15,136.45,True,False,1.016510,NaN
2024-01-01 09:23:00,136.20,137.90,133.70,134.40,376750,4619250,135.92,134.25,134.55,False,True,0.668767,NaN


In [59]:
ps.om.closed_orders

[buy, at:2024-01-01 09:32:00, b:132.4, sqat:2024-01-01 09:39:00, s:125.78, pnl:-5.0,
 buy, at:2024-01-01 10:12:00, b:138.5, sqat:2024-01-01 10:21:00, s:131.575, pnl:-5.0,
 buy, at:2024-01-01 10:23:00, b:131.35, sqat:2024-01-01 10:31:00, s:137.9175, pnl:5.0,
 buy, at:2024-01-01 10:47:00, b:105.35, sqat:2024-01-01 10:50:00, s:100.0825, pnl:-5.0,
 buy, at:2024-01-01 12:02:00, b:140.85, sqat:2024-01-01 12:06:00, s:133.8075, pnl:-5.0,
 buy, at:2024-01-01 12:47:00, b:95.95, sqat:2024-01-01 12:50:00, s:91.1525, pnl:-5.0,
 buy, at:2024-01-01 12:51:00, b:94.25, sqat:2024-01-01 13:03:00, s:89.5375, pnl:-5.0,
 buy, at:2024-01-01 13:06:00, b:91.25, sqat:2024-01-01 13:16:00, s:86.6875, pnl:-5.0,
 buy, at:2024-01-01 13:27:00, b:88.2, sqat:2024-01-01 13:30:00, s:83.79, pnl:-5.0,
 buy, at:2024-01-01 13:51:00, b:85.85, sqat:2024-01-01 13:56:00, s:81.55749999999999, pnl:-5.0,
 buy, at:2024-01-01 13:59:00, b:76.2, sqat:2024-01-01 14:02:00, s:80.01, pnl:5.0,
 buy, at:2024-01-01 14:03:00, b:88.8, sqat:2024

In [67]:
pcs = [order.pnl_pc for order in ps.om.closed_orders]
print(f"PnL Per Order: {sum(pcs) / len(pcs)}, success rate: {len([x for x in pcs if x > 0]) * 100 / len(pcs)}%")
pcs

call_orders = [order for order in ps.om.closed_orders if order.meta['type'] == 2]
[print(f"{order.created_at}, {order.pnl_pc}") for order in call_orders]

PnL Per Order: -2.5, success rate: 25.0%
2024-01-01 13:59:00, 5.0
2024-01-01 14:03:00, -5.0
2024-01-01 14:24:00, -5.0
2024-01-01 14:51:00, -5.0


[None, None, None, None]

In [63]:
i = 1
print(ps.om.closed_orders[i].meta)
print(ps.om.closed_orders[i])
print(f"Buy: {ps.om.closed_orders[i].limit_price}, Sell: {ps.om.closed_orders[i].square_off_price}, trsi: {ps.t1_ticks.loc[ps.om.closed_orders[i].created_at].rsi}, orsi: {ps.t2_ticks.loc[ps.om.closed_orders[i].created_at].rsi}")

{'atm_strike': 21700, 'tp': 6.925000000000001, 'sl': 6.925000000000001, 'type': 1}
buy, at:2024-01-01 10:12:00, b:138.5, sqat:2024-01-01 10:21:00, s:131.575, pnl:-5.0
Buy: 138.5, Sell: 131.575, trsi: 38.664863809824475, orsi: 57.25272745769357


# Call Trading Chart

In [12]:
# call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=ps.om.closed_orders[i].meta['call_strike'], option_type=OPTION_TYPE_CALL)
# put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=ps.om.closed_orders[i].meta['put_strike'], option_type=OPTION_TYPE_PUT)

call_orders = [order for order in ps.om.closed_orders if order.meta['type'] == 1]
put_orders = [order for order in ps.om.closed_orders if order.meta['type'] == 2]
call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].call_trading, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].put_monitoring, option_type=OPTION_TYPE_PUT)

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, BooleanFilter

charts, cds = ut.create_candlestick_plot(call_df, title='Call', plot=False, width=1150, height=600)
p = charts[0][0]
entry_view = CDSView(filter=IndexFilter(call_df.index.get_indexer([order.created_at for order in call_orders])))
exit_view = CDSView(filter=IndexFilter(call_df.index.get_indexer([order.square_off_at for order in call_orders])))
p.scatter(x='date_time', y='close', view=entry_view, source=cds, color='yellow', size=8, marker='triangle')
p.scatter(x='date_time', y='close', view=exit_view, source=cds, color='yellow', size=8, marker='inverted_triangle')
charts[0][0] = p
# layout = gridplot(charts)
# show(layout)
ctsad = charts
charts, cds = ut.create_candlestick_plot(put_df, title='Put', plot=False, width=1150, height=600)
p = charts[0][0]
p.scatter(x='date_time', y='close', view=entry_view, source=cds, color='fuchsia', size=8, marker='triangle')
p.scatter(x='date_time', y='close', view=exit_view, source=cds, color='fuchsia', size=8, marker='inverted_triangle')
charts[0][0] = p
charts[0].append(ctsad[0][0])
charts[1].append(ctsad[1][0])
layout = gridplot(charts)
show(layout)

/tmp/ipykernel_7779/284556689.py:29: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout = gridplot(charts)


# Put Trading Chart

In [23]:
put_orders

[buy, at:2024-01-01 11:31:00, b:21.2, sqat:2024-01-01 11:57:00, s:21.5, pnl:1.42,
 buy, at:2024-01-01 12:41:00, b:19.85, sqat:2024-01-01 12:45:00, s:18.8, pnl:-5.29,
 buy, at:2024-01-01 13:59:00, b:15.0, sqat:2024-01-01 14:01:00, s:14.5, pnl:-3.33,
 buy, at:2024-01-01 14:03:00, b:13.9, sqat:2024-01-01 14:13:00, s:13.6, pnl:-2.16,
 buy, at:2024-01-01 14:51:00, b:12.1, sqat:2024-01-01 14:56:00, s:12.0, pnl:-0.83]

In [24]:
call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].call_monitoring, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=train_dates.iloc[0].expiry, cur_dt=train_dates.iloc[0].name.date(), strike_price=train_dates.iloc[0].put_trading, option_type=OPTION_TYPE_PUT)


put_orders = [order for order in ps.om.closed_orders if order.meta['otype'] == OPTION_TYPE_PUT]

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, BooleanFilter

charts, cds = ut.create_candlestick_plot(call_df, title='Call', plot=False, width=1150, height=600)
p = charts[0][0]
entry_view = CDSView(filter=IndexFilter(call_df.index.get_indexer([order.created_at for order in put_orders])))
exit_view = CDSView(filter=IndexFilter(call_df.index.get_indexer([order.square_off_at for order in put_orders])))
p.scatter(x='date_time', y='close', view=entry_view, source=cds, color='fuchsia', size=8, marker='triangle')
p.scatter(x='date_time', y='close', view=exit_view, source=cds, color='fuchsia', size=8, marker='inverted_triangle')
charts[0][0] = p
# layout = gridplot(charts)
# show(layout)
ctsad = charts
charts, cds = ut.create_candlestick_plot(put_df, title='Put', plot=False, width=1150, height=600)
p = charts[0][0]
p.scatter(x='date_time', y='close', view=entry_view, source=cds, color='yellow', size=8, marker='triangle')
p.scatter(x='date_time', y='close', view=exit_view, source=cds, color='yellow', size=8, marker='inverted_triangle')
charts[0][0] = p
charts[0].append(ctsad[0][0])
charts[1].append(ctsad[1][0])
layout = gridplot(charts)
show(layout)

/tmp/ipykernel_7779/4247273156.py:28: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout = gridplot(charts)


# Analyze

In [6]:
instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")

SYMBOL = "NIFTY"
expiry = dt.datetime.strptime("2024-03-21", "%Y-%m-%d").date()
otype = "CE"
strike = 22300
date = dt.datetime.strptime("2024-03-15", "%Y-%m-%d").date()

tdf = ut.get_ticks(symbol=SYMBOL, expiry=expiry, strike=strike, otype=otype, date=date)
tdf = tdf.iloc[:4000]


In [7]:
# tdf.last_price.hvplot()
import talib as ta


def get_scaled_series(ser, mn, rg):
    mx = mn + rg
    scaled_series = (((ser - ser.min()) / (ser.max() - ser.min())) * (mx - mn)) + mn
    return scaled_series

tdf['cum_volume_price'] = (tdf['last_price'] * tdf['volume']).rolling(window=10).sum()
tdf['cum_volume'] = tdf['volume'].rolling(window=10).sum()
tdf['vwap'] = tdf['cum_volume_price'] / tdf['cum_volume']
tdf["adj_vol"] = ta.SMA(tdf.volume, timeperiod=10)
tdf["adj_vol_diff"] = tdf.adj_vol.diff()
tdf["sc_adj_vol_diff"] = get_scaled_series(tdf.adj_vol_diff, tdf.last_price.mean(), 100)
tdf["ema_adj_vol"] = ta.EMA(tdf.adj_vol_diff, timeperiod=30)
tdf["ema_adj_vol"] = get_scaled_series(tdf.ema_adj_vol, tdf.last_price.mean(), 100)
tdf["adj_vol"] = get_scaled_series(tdf.volume, tdf.last_price.mean(), 100)
settings = {
 'kind': 'line',
    # 'by': ['species'],
 'title': 'OHLC',
 'x': 'last_trade_time',
 'y': ['vwap', 'adj_vol']
}
# hvexplorer = tdf.hvplot.explorer(**settings, height=1000)
# hvexplorer

tdf.hvplot(
    height=1000,
    kind='line',
    responsive=False,
    title='OHLC',
    width=2000,
    x='last_trade_time',
    y=['vwap', 'ema_adj_vol', 'sc_adj_vol_diff', 'adj_vol'],
    legend='bottom_right',
    widget_location='bottom',
)

HoloViews(NdOverlay, height=1000, sizing_mode='fixed', widget_location='bottom', width=2000)